In [2]:
pip install pandas scikit-learn numpy Pillow keras_tuner


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

# Define the root directory where your image folders are located
root_directory = "PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path'])
        image_array = np.array(image.resize((256, 256)))  # Resize image to fit model input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)

Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 256, 256, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 256, 256, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 256, 256, 3) (6909,)
Ischaemia Validation set shape: (2220, 256, 256, 3) (2220,)
Ischaemia Test set shape: (741, 256, 256, 3) (741,)
Infection Training set shape: (4123, 256, 256, 3) (4123,)
Infection Validation set shape: (1325, 256, 256, 3) (1325,)
Infection Test set shape: (442, 256, 256, 3) (442,)


In [4]:
import numpy as np
from keras_tuner import HyperModel, HyperParameters
import keras_tuner as kt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, TimeDistributed, LSTM
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, SGD, RMSprop

# Define the model-building function
def build_model(hp):
    model = Sequential()
    
    # First Convolutional Block
    model.add(Conv2D(filters=hp.Int('conv_1_filters', min_value=32, max_value=64, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(256, 256, 3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    # Second Convolutional Block
    model.add(Conv2D(filters=hp.Int('conv_2_filters', min_value=64, max_value=128, step=16),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    # Third Convolutional Block
    model.add(Conv2D(filters=hp.Int('conv_3_filters', min_value=128, max_value=256, step=16),
                     kernel_size=hp.Choice('conv_3_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(BatchNormalization())
    
    model.add(TimeDistributed(Flatten()))
    
    # LSTM Layers
    model.add(LSTM(hp.Int('lstm_units_1', min_value=80, max_value=120, step=10), 
                   dropout=hp.Float('dropout_1', min_value=0.2, max_value=0.3, step=0.1), 
                   return_sequences=True))
    model.add(LSTM(hp.Int('lstm_units_2', min_value=50, max_value=70, step=10), 
                   dropout=hp.Float('dropout_2', min_value=0.2, max_value=0.3, step=0.1), 
                   return_sequences=True))
    model.add(LSTM(hp.Int('lstm_units_3', min_value=20, max_value=40, step=10), 
                   dropout=0.2, 
                   return_sequences=False))
    
    # Fully Connected Layers
    model.add(Dense(hp.Int('dense_1_units', min_value=128, max_value=512, step=64), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.3, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    
    model.add(Dense(hp.Int('dense_2_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.2, max_value=0.4, step=0.1)))
    model.add(BatchNormalization())
    
    model.add(Dense(3, activation='softmax'))
    
    # Optimizer
    learning_rate_params = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer_params = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'Adadelta', 'Adagrad'])
    
    if optimizer_params == 'adam':
        optimizer = Adam(learning_rate=learning_rate_params)
    elif optimizer_params == 'sgd':
        optimizer = SGD(learning_rate=learning_rate_params)
    elif optimizer_params == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adagrad':
        optimizer = Adagrad(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adadelta':
        optimizer = Adadelta(learning_rate=learning_rate_params)
    else:
        raise ValueError(f"Invalid optimizer choice: {optimizer_choice}")
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [5]:
import keras_tuner as kt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define HyperModel
class CNNHyperModel(HyperModel):
    def build(self, hp):
        return build_model(hp)

# Instantiate the tuner
tuner = kt.BayesianOptimization(
    hypermodel=CNNHyperModel(),
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='bayesian_tuning',
    project_name='dfu_classification'
)

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Run the search for the Ischaemia dataset
tuner.search(X_train_ischaemia, y_train_ischaemia, epochs=50, validation_data=(X_val_ischaemia, y_val_ischaemia), callbacks=[reduce_lr, early_stopping])

# Get the optimal hyperparameters and build the model
best_hp_ischaemia = tuner.get_best_hyperparameters()[0]
model_ischaemia = build_model(best_hp_ischaemia)

# Train the best model
history_ischaemia = model_ischaemia.fit(X_train_ischaemia, y_train_ischaemia, epochs=50, validation_data=(X_val_ischaemia, y_val_ischaemia), callbacks=[reduce_lr, early_stopping])

# Evaluate the model
test_loss_ischaemia, test_accuracy_ischaemia = model_ischaemia.evaluate(X_test_ischaemia, y_test_ischaemia)
print(f"Ischaemia Test Accuracy: {test_accuracy_ischaemia}")

2024-07-31 13:25:00.313793: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22066 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:42:00.0, compute capability: 8.9
2024-07-31 13:25:00.315392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22066 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:61:00.0, compute capability: 8.9
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Reloading Tuner from bayesian_tuning/dfu_classification/tuner0.json
Epoch 1/50


2024-07-31 13:25:07.594111: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
W0000 00:00:1722432307.700794  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.747283  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.748183  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.748999  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.749806  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.750616  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.756910  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432307.758265  107422 gpu_t

  3/216 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - accuracy: 0.2101 - loss: 2.2910 

W0000 00:00:1722432310.110102  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.138384  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.147310  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.154360  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.156270  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.158058  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.167875  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.169692  107422 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432310.171537  107422 gp

...

W0000 00:00:1722432319.955722  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.956848  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.957636  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.958407  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.959140  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.959891  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.961040  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.962196  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432319.963352  107423 gp

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5873 - loss: 1.0580

W0000 00:00:1722432320.574769  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.602409  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.611141  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.617601  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.619482  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.621134  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.622744  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.624393  107423 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432320.626074  107423 gp

216/216 ━━━━━━━━━━━━━━━━━━━━ 20s 62ms/step - accuracy: 0.5879 - loss: 1.0569 - val_accuracy: 0.7275 - val_loss: 0.8187 - learning_rate: 2.1704e-04
Epoch 2/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.8179 - loss: 0.5496 - val_accuracy: 0.8473 - val_loss: 0.4963 - learning_rate: 2.1704e-04
Epoch 3/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.8414 - loss: 0.4077 - val_accuracy: 0.8550 - val_loss: 0.3748 - learning_rate: 2.1704e-04
Epoch 4/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.8554 - loss: 0.3858 - val_accuracy: 0.8721 - val_loss: 0.3826 - learning_rate: 2.1704e-04
Epoch 5/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.8667 - loss: 0.3479 - val_accuracy: 0.8459 - val_loss: 0.4450 - learning_rate: 2.1704e-04
Epoch 6/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.8785 - loss: 0.3290 - val_accuracy: 0.7968 - val_loss: 0.5107 - learning_rate: 2.1704e-04
Epoch 7/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accurac

W0000 00:00:1722432723.535187  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.535875  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.536276  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.536681  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.537084  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.537489  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.537952  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.538386  107426 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722432723.538858  107426 gp

In [6]:
import keras_tuner as kt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define HyperModel
class CNNHyperModel(HyperModel):
    def build(self, hp):
        return build_model(hp)

# Instantiate the tuner
tuner = kt.BayesianOptimization(
    hypermodel=CNNHyperModel(),
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='bayesian_infection_tuning',
    project_name='dfu_classification'
)

In [7]:
# Repeat the process for the Infection dataset
tuner.search(X_train_infection, y_train_infection, epochs=50, validation_data=(X_val_infection, y_val_infection), callbacks=[reduce_lr, early_stopping])

# Get the optimal hyperparameters and build the model
best_hp_infection = tuner.get_best_hyperparameters()[0]
model_infection = build_model(best_hp_infection)

# Train the best model
history_infection = model_infection.fit(X_train_infection, y_train_infection, epochs=50, validation_data=(X_val_infection, y_val_infection), callbacks=[reduce_lr, early_stopping])

# Evaluate the model
test_loss_infection, test_accuracy_infection = model_infection.evaluate(X_test_infection, y_test_infection)
print(f"Infection Test Accuracy: {test_accuracy_infection}")


Trial 5 Complete [00h 00m 36s]
val_accuracy: 0.6045283079147339

Best val_accuracy So Far: 0.6573584675788879
Total elapsed time: 00h 03m 50s
Epoch 1/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step - accuracy: 0.4423 - loss: 1.3762 - val_accuracy: 0.4128 - val_loss: 1.2924 - learning_rate: 1.1104e-04
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.4644 - loss: 1.2506 - val_accuracy: 0.3970 - val_loss: 1.2335 - learning_rate: 1.1104e-04
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.5255 - loss: 1.1114 - val_accuracy: 0.5200 - val_loss: 1.0915 - learning_rate: 1.1104e-04
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.5455 - loss: 1.0447 - val_accuracy: 0.5955 - val_loss: 0.9224 - learning_rate: 1.1104e-04
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.5958 - loss: 0.9563 - val_accuracy: 0.6264 - val_loss: 0.8460 - learning_rate: 1.1104e-04
10/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4504 - loss: 1.264

W0000 00:00:1722433365.057732  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.059911  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.061050  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.061867  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.063029  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.064211  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.065741  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.067761  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.070319  107418 gp

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.4455 - loss: 1.2638
Infection Test Accuracy: 0.438914030790329


W0000 00:00:1722433365.294998  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.296121  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.298059  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.299206  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.300404  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.301619  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.303057  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.304533  107418 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1722433365.305912  107418 gp